<a href="https://colab.research.google.com/github/wiz124/chem169-git/blob/main/Li_Harry_RID_014_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#Exercise 0
import pickle
from Bio import SeqIO

seq_embed={}
with open('toy_esm2_embeddings.pkl', 'rb') as f:
  data=pickle.load(f)
  for seq_record in SeqIO.parse("toy_sequences.fasta", "fasta"):
      seq_embed[str(seq_record.seq)]=data[seq_record.id]
print(seq_embed)
      # for key, value in data.items():



{'MACHINELEARNING': {'residue_embeddings': array([[ 0.05606826, -0.02779197,  0.1984435 , ...,  0.29308623,
        -0.0005211 , -0.001761  ],
       [ 0.01368738,  0.08926699,  0.07153449, ...,  0.02676891,
        -0.05958881,  0.08751322],
       [ 0.0791374 ,  0.05739338,  0.05229965, ..., -0.12164824,
        -0.2122045 , -0.03057704],
       ...,
       [-0.07602476,  0.17316677,  0.04309352, ..., -0.17556725,
         0.18471679,  0.06163794],
       [ 0.0032055 ,  0.07336932,  0.23836282, ..., -0.11572927,
        -0.1422069 ,  0.17654972],
       [ 0.1213042 ,  0.09664907,  0.15444879, ..., -0.22865772,
        -0.07566334, -0.07008101]], dtype=float32), 'protein_embedding': array([ 2.7157145e-02,  6.7918792e-02,  1.1835212e-01, ...,
       -3.1786315e-02,  6.4118205e-05,  1.0375285e-02], dtype=float32)}, 'MGCHINELEARNING': {'residue_embeddings': array([[-0.01221433, -0.04243858,  0.23617591, ...,  0.26975685,
        -0.10064612,  0.03309401],
       [ 0.1237796 , -0.01890814